In [2]:
!pip install opera-utils

In [3]:
import opera_utils
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import numpy as np
import h5py
from fft_unwrap import unwrap_phase_fft
import matplotlib.pyplot as plt
import contextily as ctx
import h5py

## Interferometry steps from NASA CSLCs

Each pixel in a complex SAR image contains both amplitude and phase information:​

* **Amplitude**: Reflects the strength of the radar return signal, indicating how much energy was backscattered from the surface.​
* **Phase**: Represents the position of the wave cycle at the time of measurement, which is related to the distance between the sensor and the target on the ground.

### 1. Calculating the Interferogram:

To compute an interferogram, we multiply each pixel of the reference SAR image $S_1$ by the complex conjugate of the corresponding pixel in the secondary SAR image $S_2$:

$$ \text{Interferogram} = S_1 \times S_2^* $$

#### What this operation entails:

- **Complex Conjugate $S_2^*$**: This involves inverting the sign of the imaginary component of $S_2$. If a pixel value in $S_2$ is represented as $a + bi$ (where $a$ is the real part and $bi$ is the imaginary part), its complex conjugate is $a - bi$.

- **Multiplication**: By multiplying $S_1$ by $S_2^*$, the amplitudes are combined, and the phases are subtracted:

$$ (a_1 + b_1 i) \times (a_2 - b_2 i) = (a_1 a_2 + b_1 b_2) + (b_1 a_2 - a_1 b_2) i $$

This operation results in a new complex value for each pixel, where the **phase** component represents the difference in phase between the two original images. This phase difference corresponds to changes in the distance between the ground and the radar sensor between the two acquisition times, which can be due to ground displacement, topography, or atmospheric effects.

* The real part ($a$) represents the in-phase component of the backscattered signal
* The imaginary part ($bi$) represents the quadrature (or out-of-phase) component

Together, they encode both the amplitude and phase of the radar return

#### Extracting the Phase (Wrapped Phase):

The interferogram contains complex values, but for many applications, we are primarily interested in the **phase** information. The `np.angle` function is used to extract the **phase angle** (in radians) from each complex value, resulting in the "wrapped phase". This phase is constrained between $-\pi$ and $\pi$ and represents the modulo $2\pi$ phase difference between the two images.

- **Mathematically**, the `np.angle` function computes the phase $\theta$ of a complex number $z = a + bi$ as:

$$ \theta = \arg(z) = \tan^{-1}\left( \frac{b}{a} \right) $$

Where $\arg(z)$ is the angle of the complex number $z$ with respect to the positive real axis.

​In InSAR, the phase angle extracted from the interferogram indicates the difference in radar signal phase between two acquisitions, reflecting surface displacement. To accurately determine this angle, $\tan^{-1}$ computes the arctangent of the ratio of the imaginary to the real part of a complex number, considering their signs to place the angle in the correct quadrant. 

The "quadrant" relates directly to the Line of Sight (LOS) deformation:

In the context of SAR interferometry, the quadrant represents the direction of ground movement relative to the radar sensor. The four quadrants correspond to different LOS displacement scenarios:

- Positive phase (moving towards the sensor)
- Negative phase (moving away from the sensor)
- Minimal or no movement
- Complex displacement patterns

The phase angle calculation 

$$ \tan^{-1}\left( \frac{b}{a} \right) $$

determines the precise location within these quadrants by considering:

- The sign of the real part ($a$)
- The sign of the imaginary part ($b$)

## Extracting the Phase (Wrapped Phase):

The interferogram contains complex values, but for many applications, we are primarily interested in the phase information. The `np.angle` function is used to extract the phase angle (in radians) from each complex value, resulting in the "wrapped phase". This phase is constrained between $-\pi$ and $\pi$ and represents the modulo $2\pi$ phase difference between the two images.

## Quadrant Correction Method:

- If $a > 0$: Standard arctangent calculation
- If $a < 0$ and $b \geq 0$: Add $\pi$ to the result
- If $a < 0$ and $b < 0$: Subtract $\pi$ from the result
- If $a = 0$:
  - $b > 0$: $\frac{\pi}{2}$
  - $b < 0$: $-\frac{\pi}{2}$

This approach ensures that the phase angle accurately represents the Line of Sight (LOS) displacement, accounting for the full range of possible ground movements between two SAR acquisitions.

### 2. Coherence Calculation:

Coherence is a measure of the similarity or consistency between two SAR images. In interferometry, it can be computed using the complex values of two SAR images, typically representing the reference and secondary acquisitions.

Given two SAR images, $S_1$ (reference) and $S_2$ (secondary), the interferometric coherence $\gamma$ can be calculated using the following formula:

$$ \gamma = \frac{\left| \langle S_1 \cdot S_2^* \rangle \right|}{\sqrt{\langle |S_1|^2 \rangle \cdot \langle |S_2|^2 \rangle}} $$

Where:

- $S_1$ and $S_2$ are the complex-valued SAR images.
- $S_2^*$ is the complex conjugate of $S_2$.
- $\langle \cdot \rangle$ represents averaging over a window of pixels
- $|S_1 \cdot S_2^*|$ is the magnitude of the cross-product between $S_1$ and the complex conjugate of $S_2$, which corresponds to the interferogram.
- $|S_1|^2$ and $|S_2|^2$ are the squared magnitudes of the individual images, representing their amplitudes.

### 3. Steps to Compute Coherence:

1. **Calculate the Interferogram**: Multiply the reference image $S_1$ by the complex conjugate of the secondary image $S_2$:

   $$ \text{Interferogram} = S_1 \times S_2^* $$

   This results in a complex-valued image, with the phase difference stored in the complex argument.

2. **Magnitude of the Interferogram**: The numerator of the coherence equation involves calculating the magnitude of the cross-product:

   $$ \left| \langle S_1 \cdot S_2^* \rangle \right| $$

   This represents the degree of similarity between the two images. If the images are highly coherent (similar), this magnitude will be large.

3. **Compute Squared Magnitudes of Each Image**: The denominator involves computing the squared magnitudes of the individual images, which represent the signal strength or backscatter in each image:

   $$ \langle |S_1|^2 \rangle \quad \text{and} \quad \langle |S_2|^2 \rangle $$

4. **Compute the Coherence**: Finally, coherence is the ratio of the magnitude of the cross-product to the square root of the product of the squared magnitudes of each image.

   $$ \gamma = \frac{\left| \langle S_1 \cdot S_2^* \rangle \right|}{\sqrt{\langle |S_1|^2 \rangle \cdot \langle |S_2|^2 \rangle}} $$

> NOTE
> Coherence ranges from 0 to 1:
> - **Coherence = 1**: Perfect correlation (the two images are identical or show no change).
> - **Coherence = 0**: No correlation (the two images are unrelated or have significant differences in backscatter).

In [11]:
%%time
# load in our wrapped phase data
# NOTE: these are NASA OPERA CSLC products, same Sentinel-1 burst 12 days apart
# coregistered and corrected for topographic, atmospheric, orbital, and random phase noise
fn_1 = "/home/jehayes/gda_final/opera/cslc/OPERA_L2_CSLC-S1_T124-264305-IW3_20180408T043041Z_20240429T045313Z_S1A_VV_v1.1.h5"
fn_2 = "/home/jehayes/gda_final/opera/cslc/OPERA_L2_CSLC-S1_T124-264305-IW3_20180420T043041Z_20240429T081305Z_S1A_VV_v1.1.h5"

dsR = xr.open_dataset(fn_1,
                      group='data',
                      engine='h5netcdf')['VV'].rename(dict(x_coordinates='x', y_coordinates='y'))
tR = pd.to_datetime(xr.open_dataset(fn_1, 
                                    group='identification')['zero_doppler_start_time'].data.astype('U')) # Unicode string format
dsS = xr.open_dataset(fn_2,
                      group='data',
                      engine='h5netcdf')['VV'].rename(dict(x_coordinates='x', y_coordinates='y'))

tS = pd.to_datetime(xr.open_dataset(fn_2, group='identification')['zero_doppler_start_time'].data.astype('U'))

ifg = dsR * np.conj(dsS)
ifg.attrs['reference'] = tR
ifg.attrs['secondary'] = tS
da = xr.apply_ufunc(np.angle, ifg)
ds = da.to_dataset(name='phase')

with h5py.File(fn_1, 'r') as f:
    proj_ds = f['/data/projection']
    proj_value = proj_ds[()]
    print("Projection:", proj_value)
    for key, value in proj_ds.attrs.items():
        print(f"{key}: {value}")

ds.rio.write_crs(proj_value, inplace=True)

Projection: 32605
description: b'Projection system'
ellipsoid: b'WGS84'
epsg_code: 32605
grid_mapping_name: b'universal_transverse_mercator'
inverse_flattening: 298.257223563
semi_major_axis: 6378137.0
spatial_ref: b'PROJCS["WGS 84 / UTM zone 5N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-153],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32605"]]'
utm_zone_number: 5
CPU times: user 6.52 s, sys: 1.32 s, total: 7.83 s
Wall time: 7.95 s


<xarray.Dataset> Size: 342MB
Dimensions:      (x: 18558, y: 4602)
Coordinates:
  * x            (x) float64 148kB 2.659e+05 2.659e+05 ... 3.587e+05 3.587e+05
  * y            (y) float64 37kB 2.179e+06 2.179e+06 ... 2.133e+06 2.133e+06
    spatial_ref  int64 8B 0
Data variables:
    phase        (y, x) float32 342MB nan nan nan nan nan ... nan nan nan nan

In [12]:
def print_h5_structure(filename):
    with h5py.File(filename, 'r') as f:
        def print_groups(name):
            print(name)
        f.visit(print_groups)

# Check structure of first file
print("Structure of first file:")
print_h5_structure(fn_1)

Structure of first file:
complex64
data
data/VV
data/azimuth_carrier_phase
data/flattening_phase
data/projection
data/x_coordinates
data/x_spacing
data/y_coordinates
data/y_spacing
identification
identification/absolute_orbit_number
identification/bounding_polygon
identification/burst_id
identification/instrument_name
identification/is_geocoded
identification/look_direction
identification/mission_id
identification/orbit_pass_direction
identification/processing_center
identification/processing_date_time
identification/product_level
identification/product_specification_version
identification/product_type
identification/product_version
identification/radar_band
identification/track_number
identification/zero_doppler_end_time
identification/zero_doppler_start_time
metadata
metadata/calibration_information
metadata/calibration_information/azimuth_time
metadata/calibration_information/beta_naught
metadata/calibration_information/dn
metadata/calibration_information/gamma
metadata/calibration_

In [13]:
def get_burst_info(filename):
    with h5py.File(filename, 'r') as f:
        burst_info = f['identification/burst_id'][()]
        return burst_info
burst_id_1 = get_burst_info(fn_1)
burst_id_2 = get_burst_info(fn_2)
print(f"Burst ID for first file: {burst_id_1}")
print(f"Burst ID for second file: {burst_id_2}")

Burst ID for first file: b't124_264305_iw3'
Burst ID for second file: b't124_264305_iw3'


Burst IDs of interest for SLCs over Kilauea (4 bursts):
* T124_264305_IW2 (Ascending, southwest)
* T124_264306_IW2 (Ascending, northwest)
* T087_185683_IW1 (Descending, southeast)
* T087_185682_IW1 (Descending, northeast)

Frame IDs of interest for DISP samples over Kilauea (4 bursts):
* 33039 (Ascending, north)
* 33038 (Ascending, south)
* 23210 (Descending, north)
* 23211 (Descending, south)

In [21]:
#def extract_burst_str(burst_bytes):
#    burst_str = str(burst_bytes)
#    return burst_str.split('_')[1]
#burst_id = extract_burst_str(burst_id_1)

In [22]:
#json_file = opera_utils.datasets.fetch_burst_to_frame_mapping_file()
#js = opera_utils.read_zipped_json(json_file)

In [28]:
opera_utils.get_frame_ids_for_burst(burst_id_1.decode())

[33038, 33039]

In [30]:
!aws s3 cp s3://opera-pst-rs-pop1/products/DISP_S1/ . --recursive --no-sign-request --exclude "*" --include "*F33039*v1.1*"

fatal error: Unable to locate credentials
